In [1]:

url = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N"
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Members = soup.findAll(class_ = 'Members--bio u-richtext')
state_district = soup.findAll(class_ = 'Hero-title')
Money = soup.findAll(class_ = 'Members--number')


#### Get candidate and affiliation info 

In [2]:
def get_candidate_info(Members):
    members_list = []
    member_affiliation = []

    for i in range(len(Members)):
        if i == 0:
            name_affiliation = Members[0].text
            members_list.append(
                re.split(r"[\t\n()]", name_affiliation)[2].strip()
            )
            member_affiliation.append(
                re.split(r"[\t\n()]", name_affiliation)[3].strip()
            )
        else:
            
            name_affiliation = Members[i].text
            members_list.append(
                re.split(r"[\t()]", name_affiliation)[10].strip()
                )
            member_affiliation.append(
                re.split(r"[\t()]", name_affiliation)[11].strip()
                )
    return members_list, member_affiliation


### Return 1 for incumbent = True and 0 if incumbent = False

In [3]:
def get_incumbent(Members):    
    list = []
    for th in Members:
     #print(th.text.strip())
        list.append(re.split(r"[\t\n()]", th.text))
    incumbent_list = []
    for i in range(len(list)):
        if any("Incumbent" in item for item in list[i]):
            incumbent_list.append(1)
        else:
            incumbent_list.append(0)
    return incumbent_list

#### Get percentage of votes obtained

In [4]:
def get_percentage(Members):
    list = []
    for th in Members:
     #print(th.text.strip())
        list.append(re.split(r"[()]", th.text))
    incumbent_list = []
    for i in range(len(list)):
        incumbent_list.append(
            list[i][3]
            )
    return incumbent_list

In [5]:
def get_winner(Members):
    percent_num = []
    for th in Members:
        percent_num.append(re.split(r"[(%)]", th.text)[3])
    winner_list = []
    for i in range(len(percent_num)):
        if percent_num[i] == max(percent_num):
            winner_list.append(1)
        else:
            winner_list.append(0)
    return winner_list




In [6]:
def get_state_district(state_district):
    state_list = []
    district_list = []
    for i in range(len(Members)):
        state_list.append(state_district[0].text.split()[0])
        district_list.append(state_district[0].text.split()[2])

    return state_list, district_list

In [7]:
def get_money_details(Money):
    Money_list = []
    for i in range(len(Money)):
        Money_list.append(float(re.sub(",","", Money[i].text.split("$")[1])))
    Money_raised = Money_list[0::3]
    Money_spent = Money_list[1::3]
    return Money_raised, Money_spent

In [8]:
incumbent_list = get_incumbent(Members)
percentage_list = get_percentage(Members)
winner_list = get_winner(Members)
candidate_list = get_candidate_info(Members)
state_list = get_state_district(state_district)[0]
district_list = get_state_district(state_district)[1]
Money_details = get_money_details(Money)

In [9]:
candidate_df = pd.DataFrame({"Candidate Name" : candidate_list[0],
                             "Affiliation" : candidate_list[1],
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_details[0],
                             "Money Spent": Money_details[1]}
                             )

In [10]:
candidate_df

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
0,Mark Green,R,1,69.9% of vote,1,Tennessee,07,1194960.0,935487.0
1,Kiran Sreepada,D,0,27.3% of vote,0,Tennessee,07,206644.0,207191.0
2,Ronald Brown,I,0,2.2% of vote,0,Tennessee,07,1750.0,0.0
3,Scott Vieira Jr,I,0,0.6% of vote,0,Tennessee,07,655.0,1049.0


In [11]:

url_list = []
for i in range(1,10):
    url_list.append(f"https://www.opensecrets.org/races/candidates?cycle=2020&id=TN0{i}&spec=N")
url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N']

#### Looping through districts 

In [20]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
affiliation_list = []
incumbent_list = []
percentage_list = []
winner_list = []
candidate_list = []
state_list = []
district_list = []
Money_raised = []
Money_spent = []

for url in url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    Members = soup.findAll(class_ = 'Members--bio u-richtext')
    state_district = soup.findAll(class_ = 'Hero-title')
    Money = soup.findAll(class_ = 'Members--number')
    
    incumbent_list.extend(get_incumbent(Members))
    percentage_list.extend(get_percentage(Members))
    winner_list.extend(get_winner(Members))
    candidate_list.extend(get_candidate_info(Members)[0])
    affiliation_list.extend(get_candidate_info(Members)[1])
    state_list.extend(get_state_district(state_district)[0])
    district_list.extend(get_state_district(state_district)[1])
    Money_raised.extend(get_money_details(Money)[0])
    Money_spent.extend(get_money_details(Money)[1])


TN_df = pd.DataFrame({"Candidate Name" : candidate_list,
                             "Affiliation" : affiliation_list,
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_raised,
                             "Money Spent": Money_spent}
                             )


In [19]:
incumbent_list

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0]

In [21]:
TN_df

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
0,,,0,74.8% of vote,1,Tennessee,01,2126946.0,1869100.0
1,Blair Nicole Walsingham,D,0,22.4% of vote,0,Tennessee,01,140209.0,134995.0
2,Tim Burchett,R,1,67.7% of vote,1,Tennessee,02,1336276.0,878488.0
3,Renee Hoyos,D,0,31.0% of vote,0,Tennessee,02,812784.0,816793.0
4,Chuck Fleischmann,R,1,67.3% of vote,1,Tennessee,03,1051653.0,381411.0
5,Meg Gorman,D,0,30.5% of vote,0,Tennessee,03,85843.0,77760.0
6,Scott Desjarlais,R,1,66.7% of vote,1,Tennessee,04,331464.0,392499.0
7,Christopher Hale,D,0,33.3% of vote,0,Tennessee,04,308731.0,302996.0
8,Jim Cooper,D,1,100.0% of vote,1,Tennessee,05,936569.0,1332131.0
9,John Rose,R,1,73.7% of vote,1,Tennessee,06,1050429.0,625688.0
